# 导入包

In [1]:
#!/usr/bin/env python
# coding: utf-8
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from time import time
import shutil
import argparse
import configparser
import copy
import os
import numpy as np
import torch
import torch.utils.data
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np
from time import time
from scipy.sparse.linalg import eigs
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import math
import numpy as np
from torch.nn.utils import weight_norm
import numpy as np
from torch.autograd import Variable
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from sklearn.utils import resample

In [2]:
def MinMaxnormalization(train, test ,_max ,_min):
    
    def normalize(x):
        x = 1. * (x - _min) / (_max - _min)
        x = 2. * x - 1.
        return x

    train_norm = normalize(train)
    test_norm = normalize(test)

    return train_norm,test_norm

In [3]:
def data_generator(root, param):
    dict_table = []
    # 读取CSV文件（假设文件名格式为"root_param.csv"，例如 "/data_A320_LDGL.csv"）
    X_train = pd.read_csv(root + "_" + param + ".csv")
    
    # 提取双标签：Label1 和 Label2
    Y_labels = ["Label1", "Label2"]
    Y = torch.from_numpy(X_train[Y_labels].values)  # 形状变为 [样本数, 2]
    
    # 构建特征数据（排除所有标签列）
    for v in X_train.drop(Y_labels, axis=1).values:
        table = []
        table.append(list(v))  # 将每个样本的特征序列包装为列表
        dict_table.append(table)
    
    # 转换为PyTorch张量：X形状为 [样本数, 1, 特征长度]
    X = torch.Tensor(dict_table)
    
    return X, Y

# 获取数据

In [4]:
root = "/mnt/sdb/Dataset/LiCX/A320/origin/"
X_1HZ, X_2HZ, X_4HZ, X_8HZ, Y = [], [], [], [], []
select = 30
#选择相应的参数
param_list = ['ALT_STD','RADIO_LH','RADIO_RH','IVV','IAS','LDGNOS','PITCH','PITCH_CMD','ROLL','ROLL_CMD','RUDD',
      'WIN_CRS','WIN_ALG','N11','N12','TLA1','TLA2','VRTG']#18个参数
param_list_1HZ, param_list_2HZ, param_list_4HZ, param_list_8HZ = [], [], [], []

# 标志变量，用于确保Y只被提取一次
labels_extracted = False
for param in param_list:
    # 加载CSV文件
    df = pd.read_csv(root + param + ".csv")
    
    # 提取双标签 (Label1和Label2)，只在第一次循环时提取
    y_labels = ["Label1", "Label2"]
    if not labels_extracted:  # 如果标签尚未提取
        Y = torch.from_numpy(df[y_labels].values)  # 形状 [样本数, 2]
        labels_extracted = True  # 标记为已提取
    
    # 构建特征张量（排除Time和双标签列）
    dict_table = []
    for row in df.drop(["Time"] + y_labels, axis=1).values:
        dict_table.append([list(row)])  # 保持三维结构 [样本数, 1, 时间步]
    
    X = torch.Tensor(dict_table)
    
    # 按采样率分类处理
    time_steps = X.shape[2]
    print(f"参数: {param}, 时间步长: {time_steps}")  # 打印调试信息
    
    if time_steps == 100:       # 1Hz采样率
        cut = select
        X_1HZ.append(X[:, :, :cut])
        param_list_1HZ.append(param)
    elif time_steps == 200:    # 2Hz采样率
        cut = select * 2
        X_2HZ.append(X[:, :, :cut])
        param_list_2HZ.append(param)
    elif time_steps == 400:    # 4Hz采样率
        cut = select * 4
        X_4HZ.append(X[:, :, :cut])
        param_list_4HZ.append(param)
    elif time_steps == 800:    # 8Hz采样率
        cut = select * 8
        X_8HZ.append(X[:, :, :cut])
        param_list_8HZ.append(param)
    else:
        print(f"参数 {param} 的时间步长 {time_steps} 不符合预期采样率！")  # 打印警告信息
# 沿特征维度拼接（最终形状）
X_1HZ_tensor = torch.cat(X_1HZ, dim=1)  # [样本数, 参数数_1HZ, 30]
X_2HZ_tensor = torch.cat(X_2HZ, dim=1)  # [样本数, 参数数_2HZ, 60]
X_4HZ_tensor = torch.cat(X_4HZ, dim=1)  # [样本数, 参数数_4HZ, 120]
X_8HZ_tensor = torch.cat(X_8HZ, dim=1)  # [样本数, 参数数_8HZ, 240]

参数: ALT_STD, 时间步长: 100
参数: RADIO_LH, 时间步长: 400
参数: RADIO_RH, 时间步长: 400
参数: IVV, 时间步长: 100
参数: IAS, 时间步长: 100
参数: LDGNOS, 时间步长: 400
参数: PITCH, 时间步长: 400
参数: PITCH_CMD, 时间步长: 800
参数: ROLL, 时间步长: 200
参数: ROLL_CMD, 时间步长: 800
参数: RUDD, 时间步长: 200
参数: WIN_CRS, 时间步长: 100
参数: WIN_ALG, 时间步长: 100
参数: N11, 时间步长: 100
参数: N12, 时间步长: 100
参数: TLA1, 时间步长: 100
参数: TLA2, 时间步长: 100
参数: VRTG, 时间步长: 800


In [5]:
param_list_8HZ


['PITCH_CMD', 'ROLL_CMD', 'VRTG']

# 划分数据集

In [6]:
split_line = int(len(Y) * 0.7)


In [7]:
train_X_1HZ_tensor=X_1HZ_tensor[:split_line]
train_X_2HZ_tensor=X_2HZ_tensor[:split_line]
train_X_4HZ_tensor=X_4HZ_tensor[:split_line]
train_X_8HZ_tensor=X_8HZ_tensor[:split_line]
train_Y_tensor=Y[:split_line]

In [8]:
test_X_1HZ_tensor=X_1HZ_tensor[split_line:]
test_X_2HZ_tensor=X_2HZ_tensor[split_line:]
test_X_4HZ_tensor=X_4HZ_tensor[split_line:]
test_X_8HZ_tensor=X_8HZ_tensor[split_line:]
test_Y_tensor=Y[split_line:]

# 训练集上采样

In [9]:
# indices_0 = torch.where(train_Y_tensor == 0)[0]
# indices_1 = torch.where(train_Y_tensor == 1)[0]
# # indices_0 = torch.where(test_Y_tensor == 0)[0]
# # indices_1 = torch.where(test_Y_tensor == 1)[0]
# import torch
# from sklearn.utils import resample

# # ========== 1) 将双标签 (label1, label2) 转成单一"组合标签" ==========
# #   - 如果 label1 ∈ {0,1}, label2 ∈ {0,1},
# #   - 那么组合后有4种可能: 0 -> [0,0], 1 -> [0,1], 2 -> [1,0], 3 -> [1,1].
# #   - y_combination = label1*2 + label2
# train_Y_label1 = train_Y_tensor[:, 0]
# train_Y_label2 = train_Y_tensor[:, 1]
# y_combination  = train_Y_label1 * 2 + train_Y_label2  # shape [N], 取值 ∈ {0,1,2,3}

# # ========== 2) 分别找出这4种组合的索引并上采样 ==========
# combo_classes = [0, 1, 2, 3]  # 对应 [0,0], [0,1], [1,0], [1,1]
# indices_list = []
# for combo in combo_classes:
#     idx = torch.where(y_combination == combo)[0]
#     indices_list.append(idx)

# # 计算各组合的样本数
# counts = [len(idxs) for idxs in indices_list]
# max_count = max(counts)

# # 分别上采样至 max_count
# upsampled_indices_list = []
# for combo_idx, combo in enumerate(combo_classes):
#     original_indices = indices_list[combo_idx]
#     # 如果某类本身就有 max_count，那么无需 resample，否则进行上采样
#     if len(original_indices) < max_count:
#         # 利用 sklearn.utils.resample 做有放回的上采样
#         upsampled = resample(
#             original_indices.numpy(),   # 先转成 numpy
#             replace=True, 
#             n_samples=max_count, 
#             random_state=42
#         )
#         upsampled = torch.from_numpy(upsampled)  # 再转回 torch.Tensor
#         upsampled_indices_list.append(upsampled)
#     else:
#         # 这类样本数 >= max_count，就不再重复
#         upsampled_indices_list.append(original_indices)

# balanced_indices = torch.cat(upsampled_indices_list, dim=0)  # [max_count*4]

# # ========== 3) 依据 balanced_indices 取出各路 X 和 Y ==========
# train_X_1HZ_balanced = train_X_1HZ_tensor[balanced_indices]
# train_X_2HZ_balanced = train_X_2HZ_tensor[balanced_indices]
# train_X_4HZ_balanced = train_X_4HZ_tensor[balanced_indices]
# train_X_8HZ_balanced = train_X_8HZ_tensor[balanced_indices]
# train_Y_balanced      = train_Y_tensor[balanced_indices]

# # ========== 4) 再做一次随机 shuffle (可选) ==========
# shuffled_indices = torch.randperm(train_X_1HZ_balanced.size(0))
# train_X_1HZ_tensor = train_X_1HZ_balanced[shuffled_indices]
# train_X_2HZ_tensor = train_X_2HZ_balanced[shuffled_indices]
# train_X_4HZ_tensor= train_X_4HZ_balanced[shuffled_indices]
# train_X_8HZ_tensor = train_X_8HZ_balanced[shuffled_indices]
# train_Y_tensor     = train_Y_balanced[shuffled_indices]

# # ========== 5) 构造 Dataset / DataLoader ==========
# train_dataset = torch.utils.data.TensorDataset(
#     train_X_1HZ_tensor, 
#     train_X_2HZ_tensor, 
#     train_X_4HZ_tensor, 
#     train_X_8HZ_tensor,
#     train_Y_tensor
# )
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
import torch
from sklearn.utils import resample

# ========== 1) 将双标签 (label1, label2) 转成单一"组合标签" ==========
#   - 如果 label1 ∈ {0,1}, label2 ∈ {0,1},
#   - 那么组合后有4种可能: 0 -> [0,0], 1 -> [0,1], 2 -> [1,0], 3 -> [1,1].
#   - y_combination = label1*2 + label2
import torch
from sklearn.utils import resample
import numpy as np

def stratified_upsample(train_X_tensors, train_Y_tensor):
    """
    对多模态输入数据进行分层上采样，平衡正负样本
    
    参数:
        train_X_tensors: 包含多个频率输入特征的元组 (X_1HZ, X_2HZ, X_4HZ, X_8HZ)
        train_Y_tensor: 标签张量，形状为 [N, 2]
        
    返回:
        平衡后的 (X_1HZ, X_2HZ, X_4HZ, X_8HZ, Y) 元组
    """
    # 解包输入张量
    train_X_1HZ_tensor, train_X_2HZ_tensor, train_X_4HZ_tensor, train_X_8HZ_tensor = train_X_tensors
    
    # ========== 1. 创建组合标签 ==========
    # 将双标签转换为单一组合标签 (4种组合)
    train_Y_label1 = train_Y_tensor[:, 0]
    train_Y_label2 = train_Y_tensor[:, 1]
    y_combination = train_Y_label1 * 2 + train_Y_label2  # 取值 ∈ {0,1,2,3}
    
    # ========== 2. 分层上采样 ==========
    combo_classes = [0, 1, 2, 3]  # 对应 [0,0], [0,1], [1,0], [1,1]
    indices_list = []
    
    # 获取每个类别的索引
    for combo in combo_classes:
        idx = torch.where(y_combination == combo)[0]
        indices_list.append(idx)
    
    # 计算各类别样本数并确定最大样本数
    counts = [len(idxs) for idxs in indices_list]
    max_count = max(counts)
    
    # 对每个类别进行上采样
    upsampled_indices_list = []
    for combo_idx, combo in enumerate(combo_classes):
        original_indices = indices_list[combo_idx]
        
        if len(original_indices) < max_count:
            # 使用分层采样进行上采样
            upsampled = resample(
                original_indices.numpy(),
                replace=True,  # 有放回抽样
                n_samples=max_count,
                stratify=y_combination[original_indices].numpy(),  # 分层信息
                random_state=42
            )
            upsampled = torch.from_numpy(upsampled)
        else:
            # 如果样本数已经足够，则随机下采样到max_count
            upsampled = original_indices[torch.randperm(len(original_indices))[:max_count]]
        
        upsampled_indices_list.append(upsampled)
    
    # 合并所有上采样后的索引
    balanced_indices = torch.cat(upsampled_indices_list, dim=0)
    
    # ========== 3. 获取平衡后的数据 ==========
    train_X_1HZ_balanced = train_X_1HZ_tensor[balanced_indices]
    train_X_2HZ_balanced = train_X_2HZ_tensor[balanced_indices]
    train_X_4HZ_balanced = train_X_4HZ_tensor[balanced_indices]
    train_X_8HZ_balanced = train_X_8HZ_tensor[balanced_indices]
    train_Y_balanced = train_Y_tensor[balanced_indices]
    
    # ========== 4. 打乱数据顺序 ==========
    shuffled_indices = torch.randperm(train_X_1HZ_balanced.size(0))
    train_X_1HZ_tensor = train_X_1HZ_balanced[shuffled_indices]
    train_X_2HZ_tensor = train_X_2HZ_balanced[shuffled_indices]
    train_X_4HZ_tensor = train_X_4HZ_balanced[shuffled_indices]
    train_X_8HZ_tensor = train_X_8HZ_balanced[shuffled_indices]
    train_Y_tensor = train_Y_balanced[shuffled_indices]
    
    return (train_X_1HZ_tensor, train_X_2HZ_tensor, 
            train_X_4HZ_tensor, train_X_8HZ_tensor, 
            train_Y_tensor)

# 使用示例
train_X_tensors = (train_X_1HZ_tensor, train_X_2HZ_tensor, 
                   train_X_4HZ_tensor, train_X_8HZ_tensor)
balanced_data = stratified_upsample(train_X_tensors, train_Y_tensor)

# 创建DataLoader
train_dataset = torch.utils.data.TensorDataset(*balanced_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)


# 测试集上采样

In [10]:
import torch
from sklearn.utils import resample
import numpy as np

def stratified_upsample_testset(test_X_tensors, test_Y_tensor):
    """
    对测试集进行分层上采样，保持与训练集相同的类别平衡策略
    
    参数:
        test_X_tensors: 元组形式的多频率测试数据 (X_1HZ, X_2HZ, X_4HZ, X_8HZ)
        test_Y_tensor: 测试集标签张量，形状 [N_test, 2]
        
    返回:
        平衡后的测试集 (X_1HZ, X_2HZ, X_4HZ, X_8HZ, Y) 元组
    """
    # 解包测试集数据
    test_X_1HZ, test_X_2HZ, test_X_4HZ, test_X_8HZ = test_X_tensors
    
    # ===== 1. 创建测试集组合标签 =====
    test_Y_label1 = test_Y_tensor[:, 0]
    test_Y_label2 = test_Y_tensor[:, 1]
    y_test_combo = test_Y_label1 * 2 + test_Y_label2  # 组合标签 ∈ {0,1,2,3}
    
    # ===== 2. 分层上采样 =====
    combo_classes = [0, 1, 2, 3]
    test_indices_list = [torch.where(y_test_combo == c)[0] for c in combo_classes]
    
    # 计算各类别样本数（保持与训练集相同的max_count）
    test_counts = [len(idxs) for idxs in test_indices_list]
    print(f"测试集原始类别分布: {dict(zip(combo_classes, test_counts))}")
    
    # 使用训练集确定的max_count（或根据测试集独立计算）
    target_count = max(test_counts)  # 或者使用训练集的max_count
    
    # 分层上采样每类
    upsampled_test_indices = []
    for i, idx in enumerate(test_indices_list):
        if len(idx) < target_count:
            # 有放回上采样（保持原始分布）
            sampled = resample(
                idx.numpy(),
                replace=True,
                n_samples=target_count,
                stratify=y_test_combo[idx].numpy(),  # 分层采样
                random_state=42
            )
            upsampled_test_indices.append(torch.from_numpy(sampled))
        else:
            # 随机选择target_count个样本（无放回）
            upsampled_test_indices.append(idx[torch.randperm(len(idx))[:target_count]])
    
    # 合并索引并打乱
    balanced_test_indices = torch.cat(upsampled_test_indices)
    shuffled_idx = torch.randperm(len(balanced_test_indices))
    
    # ===== 3. 提取平衡后的测试数据 =====
    balanced_test_data = (
        test_X_1HZ[balanced_test_indices][shuffled_idx],
        test_X_2HZ[balanced_test_indices][shuffled_idx],
        test_X_4HZ[balanced_test_indices][shuffled_idx],
        test_X_8HZ[balanced_test_indices][shuffled_idx],
        test_Y_tensor[balanced_test_indices][shuffled_idx]
    )
    
    return balanced_test_data

# 使用示例
test_X_tensors = (test_X_1HZ_tensor, test_X_2HZ_tensor, 
                 test_X_4HZ_tensor, test_X_8HZ_tensor)
balanced_test = stratified_upsample_testset(test_X_tensors, test_Y_tensor)

# 创建测试集DataLoader
test_dataset = torch.utils.data.TensorDataset(*balanced_test)
test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=256, 
    shuffle=False  # 测试集通常不shuffle
)

测试集原始类别分布: {0: 11198, 1: 125, 2: 30, 3: 22}


In [11]:
max_1HZ = torch.amax(train_X_1HZ_tensor, dim=(0, 2), keepdims=True)
min_1HZ = torch.amin(train_X_1HZ_tensor, dim=(0, 2), keepdims=True)
max_2HZ = torch.amax(train_X_2HZ_tensor, dim=(0, 2), keepdims=True)
min_2HZ = torch.amin(train_X_2HZ_tensor, dim=(0, 2), keepdims=True)
max_4HZ = torch.amax(train_X_4HZ_tensor, dim=(0, 2), keepdims=True)
min_4HZ = torch.amin(train_X_4HZ_tensor, dim=(0, 2), keepdims=True)
max_8HZ = torch.amax(train_X_8HZ_tensor, dim=(0, 2), keepdims=True)
min_8HZ = torch.amin(train_X_8HZ_tensor, dim=(0, 2), keepdims=True)

# 归一化

In [12]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# 假设下面这些变量已经定义： 
# train_X_1HZ_tensor, test_X_1HZ_tensor, train_X_2HZ_tensor, test_X_2HZ_tensor, 
# train_X_4HZ_tensor, test_X_4HZ_tensor, train_X_8HZ_tensor, test_X_8HZ_tensor,
# train_Y_tensor, test_Y_tensor
# 以及归一化所需的最大值最小值： max_1HZ, min_1HZ, max_2HZ, min_2HZ, max_4HZ, min_4HZ, max_8HZ, min_8HZ
# 还有 reshape 需要的维度：dim, dim_test, dim2, dim4, dim8, select

# 归一化函数（示例，根据实际情况修改）
def MinMaxnormalization(train_tensor, test_tensor, max_value, min_value, eps=1e-8):
    denominator = (max_value - min_value).clamp_min(eps)  # 确保分母至少为 eps
    train_norm = (train_tensor - min_value) / denominator
    test_norm  = (test_tensor - min_value) / denominator
    return train_norm, test_norm


# 设置是否进行归一化处理
apply_normalization = True

if apply_normalization:
    # 对各个频率的数据进行归一化
    train_X_1HZ_tensor_norm, test_X_1HZ_tensor_norm = MinMaxnormalization(train_X_1HZ_tensor, test_X_1HZ_tensor, max_1HZ, min_1HZ)
    train_X_2HZ_tensor_norm, test_X_2HZ_tensor_norm = MinMaxnormalization(train_X_2HZ_tensor, test_X_2HZ_tensor, max_2HZ, min_2HZ)
    train_X_4HZ_tensor_norm, test_X_4HZ_tensor_norm = MinMaxnormalization(train_X_4HZ_tensor, test_X_4HZ_tensor, max_4HZ, min_4HZ)
    train_X_8HZ_tensor_norm, test_X_8HZ_tensor_norm = MinMaxnormalization(train_X_8HZ_tensor, test_X_8HZ_tensor, max_8HZ, min_8HZ)
else:
    # 若不进行归一化，则直接使用原始数据（假设 1HZ 数据也无需归一化）
    train_X_1HZ_tensor_norm = train_X_1HZ_tensor
    test_X_1HZ_tensor_norm = test_X_1HZ_tensor
    train_X_2HZ_tensor_norm = train_X_2HZ_tensor
    test_X_2HZ_tensor_norm = test_X_2HZ_tensor
    train_X_4HZ_tensor_norm = train_X_4HZ_tensor
    test_X_4HZ_tensor_norm = test_X_4HZ_tensor
    train_X_8HZ_tensor_norm = train_X_8HZ_tensor
    test_X_8HZ_tensor_norm = test_X_8HZ_tensor


dim, dim2,dim4,dim8,dim_test = train_X_1HZ_tensor_norm.shape[0],train_X_2HZ_tensor_norm.shape[1],train_X_4HZ_tensor_norm.shape[1],train_X_8HZ_tensor_norm.shape[1],test_X_1HZ_tensor_norm.shape[0]
# 对 2HZ、4HZ、8HZ 数据进行 reshape（1HZ数据的 reshape 可根据需要添加）
train_X_2HZ_tensor_norm = train_X_2HZ_tensor_norm.reshape(dim, dim2, select, 2)
test_X_2HZ_tensor_norm = test_X_2HZ_tensor_norm.reshape(dim_test, dim2, select, 2)

train_X_4HZ_tensor_norm = train_X_4HZ_tensor_norm.reshape(dim, dim4, select, 4)
test_X_4HZ_tensor_norm = test_X_4HZ_tensor_norm.reshape(dim_test, dim4, select, 4)

train_X_8HZ_tensor_norm = train_X_8HZ_tensor_norm.reshape(dim, dim8, select, 8)
test_X_8HZ_tensor_norm = test_X_8HZ_tensor_norm.reshape(dim_test, dim8, select, 8)

# 创建 TensorDataset 和 DataLoader
train_dataset = TensorDataset(train_X_1HZ_tensor_norm,
                              train_X_2HZ_tensor_norm,
                              train_X_4HZ_tensor_norm,
                              train_X_8HZ_tensor_norm,
                              train_Y_tensor)
test_dataset = TensorDataset(test_X_1HZ_tensor_norm,
                             test_X_2HZ_tensor_norm,
                             test_X_4HZ_tensor_norm,
                             test_X_8HZ_tensor_norm,
                             test_Y_tensor)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)


# Mutli_TRBET

In [13]:
import torch
import math
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])




class MultiScale_TemporalConv(nn.Module):
    def __init__(self, in_channels, kernel_size=3, dilations=[1, 2, 4, 8], residual=True):
        super(MultiScale_TemporalConv, self).__init__()
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.dilations = dilations
        self.residual = residual

        # 创建多尺度卷积分支
        self.branches = nn.ModuleList()
        for dilation in dilations:
            # 计算保持序列长度不变所需的 padding
            padding = (kernel_size - 1) * dilation // 2
            self.branches.append(nn.Sequential(
                nn.Conv1d(
                    in_channels, in_channels, kernel_size,
                    padding=padding, dilation=dilation, groups=in_channels
                ),
                nn.BatchNorm1d(in_channels),
                nn.ReLU(inplace=True)
            ))

        # 残差连接（1x1卷积）
        if self.residual:
            self.residual_conv = nn.Conv1d(in_channels, in_channels, kernel_size=1)

    def forward(self, x):
        # x: [B, C, T]
        residual = x
        branch_outputs = [branch(x) for branch in self.branches]

        # 多尺度输出求平均
        out = sum(branch_outputs) / len(branch_outputs)

        # 残差连接
        if self.residual:
            out = out + self.residual_conv(residual)

        return out  # shape: [B, C, T]

class MultiscaleTrendAwareAttention(nn.Module):
    def __init__(self, attention_input_dim, attention_dim, context_dim,
                 K=4, kernel_size=3, dilations=(1,2,4,8),
                 local_kernel=3, diff_kernel=3, window_size=5):
        super().__init__()
        assert attention_dim % K == 0
        self.d = attention_dim // K
        self.K = K
        self.window_size = window_size

        # 1) Q,K,V 投影
        self.proj_q = nn.Linear(attention_input_dim, attention_dim)
        self.proj_k = nn.Linear(attention_input_dim, attention_dim)
        self.Wv    = nn.Linear(attention_input_dim, context_dim)

        # 2) 多尺度卷积分支
        self.scale_convs = nn.ModuleList()
        for d in dilations:
            pad = (kernel_size-1)*d//2
            self.scale_convs.append(
                nn.Conv1d(attention_dim, attention_dim, kernel_size,
                          padding=pad, dilation=d, groups=attention_dim)
            )
        # 3) 局部邻域卷积
        pad0 = (local_kernel-1)//2
        self.local_conv = nn.Conv1d(attention_dim, attention_dim,
                                    local_kernel, padding=pad0,
                                    groups=attention_dim)
        # 4) 差分卷积（高通）
        pd = (diff_kernel-1)//2
        self.diff_conv = nn.Conv1d(attention_dim, attention_dim,
                                   diff_kernel, padding=pd,
                                   groups=attention_dim)

        # 5) 各分支可学习融合权重
        self.branches = len(dilations) + 2  # 多尺度 + 局部 + 差分
        self.scale_weights = nn.Parameter(torch.ones(self.branches))

        # 6) 归一化
        self.norm_q = nn.BatchNorm1d(attention_dim)
        self.norm_k = nn.BatchNorm1d(attention_dim)

        # 7) 输出投影
        self.proj = nn.Linear(context_dim, context_dim)

    def forward(self, x):
        # x: [B, T, D_in]
        B, T, _ = x.shape

        # --- Q,K,V ---
        Q = self.proj_q(x)  # [B,T,D_att]
        K = self.proj_k(x)
        V = self.Wv(x)      # [B,T,D_ctx]

        # 转为 conv 格式
        Qc = Q.permute(0,2,1)  # [B, D_att, T]
        Kc = K.permute(0,2,1)

        # --- 各分支特征 ---
        feats = []
        for conv in self.scale_convs:
            feats.append(conv(Qc))         # 多尺度
        feats.append(self.local_conv(Qc))  # 局部
        feats.append(self.diff_conv(Qc))   # 差分

        # --- 可学习融合 ---
        w = F.softmax(self.scale_weights, dim=0)  # [branches]
        fused = sum(w[i] * feats[i] for i in range(self.branches))  # [B,D_att,T]

        # --- Norm & reshape back ---
        Qc = self.norm_q(fused)
        Q = Qc.permute(0,2,1)  # [B,T,D_att]

        # 同理处理 K
        # （为了简化，这里直接复用 fused 也可单独做一套）
        Kc = self.norm_k(fused)
        K = Kc.permute(0,2,1)

        # --- 多头拆分 ---
        q = Q.view(B, T, self.K, self.d).permute(0,2,1,3)   # [B,K,T,d]
        k = K.view(B, T, self.K, self.d).permute(0,2,3,1)   # [B,K,d,T]
        v = V.view(B, T, self.K, -1).permute(0,2,1,3)       # [B,K,T,ctx/K]

        # --- 窗口掩码 ---
        idxs = torch.arange(T, device=x.device)
        # mask[i,j] = True if j < i-window_size
        mask = (idxs[None,None,:] < (idxs[:,None] - self.window_size))
        mask = mask.unsqueeze(0).expand(B,self.K,T,T)  # [B,K,T,T]

        # --- 注意力计算 ---
        scores = (q @ k) / math.sqrt(self.d)           # [B,K,T,T]
        scores = scores.masked_fill(mask, -1e9)
        attn   = F.softmax(scores, dim=-1)
        out    = attn @ v                              # [B,K,T,ctx/K]
        out    = out.permute(0,2,1,3).reshape(B,T,-1)   # [B,T,ctx]

        # --- 输出映射 & 返回 ---
        out = self.proj(out)
        return out, attn.mean(dim=1)  # [B,T,ctx],   [B,T,T]
        
class TimeTrendAttention(nn.Module):
    def __init__(self, attention_input_dim, outchannels, attention_dim, context_dim, L, K=4):
        super().__init__()
        assert attention_dim % K == 0, f"attention_dim ({attention_dim}) must be divisible by K ({K})"
        
        self.convs = nn.Sequential(
            nn.Conv1d(attention_input_dim, outchannels, kernel_size=L, padding=L//2),
            nn.BatchNorm1d(outchannels),
            nn.ReLU()
        )
        self.attention = MultiscaleTrendAwareAttention(
            attention_input_dim=outchannels,
            attention_dim=attention_dim,
            context_dim=context_dim,
            K=K
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)  # [batch, input_dim, time_len]
        out = self.convs(x)      # [batch, outchannels, time_len]
        out = out.permute(0, 2, 1)  # [batch, time_len, outchannels]
        out, weights = self.attention(out)
        return out, weights


class MTAEncoder(nn.Module):
    def __init__(self, hidden_dim, target_len=30,
                 kernel_size=3, dilations=[1,2,4,8], residual=True):
        super(MTAEncoder, self).__init__()
        assert hidden_dim % 4 == 0, f"hidden_dim ({hidden_dim}) must be divisible by 4"
        self.hidden_dim = hidden_dim
        self.target_len = target_len

        # --- MultiScale Temporal Convs for each path ---
        # 1Hz Path: 9 channels
        self.ms_conv1 = nn.ModuleList([
            MultiScale_TemporalConv(in_channels=1,
                                    kernel_size=kernel_size,
                                    dilations=dilations,
                                    residual=residual)
            for _ in range(9)
        ])
        # 2Hz Path: 2 channels
        self.ms_conv2 = nn.ModuleList([
            MultiScale_TemporalConv(1, kernel_size, dilations, residual)
            for _ in range(2)
        ])
        # 4Hz Path: 4 channels
        self.ms_conv3 = nn.ModuleList([
            MultiScale_TemporalConv(1, kernel_size, dilations, residual)
            for _ in range(4)
        ])
        # 8Hz Path: 3 channels
        self.ms_conv4 = nn.ModuleList([
            MultiScale_TemporalConv(1, kernel_size, dilations, residual)
            for _ in range(3)
        ])

        # --- Original GRU & TimeTrendAttention blocks ---
        self.GRU1 = nn.ModuleList([nn.GRU(1, hidden_dim, batch_first=True) for _ in range(9)])
        self.TimeInter1 = nn.ModuleList([TimeTrendAttention(hidden_dim, hidden_dim, hidden_dim, hidden_dim, L=1) for _ in range(9)])

        self.GRU2 = nn.ModuleList([nn.GRU(1, hidden_dim, batch_first=True) for _ in range(2)])
        self.TimeInter2 = nn.ModuleList([TimeTrendAttention(hidden_dim, hidden_dim, hidden_dim, hidden_dim, L=3) for _ in range(2)])

        self.GRU3 = nn.ModuleList([nn.GRU(1, hidden_dim, batch_first=True) for _ in range(4)])
        self.TimeInter3 = nn.ModuleList([TimeTrendAttention(hidden_dim, hidden_dim, hidden_dim, hidden_dim, L=5) for _ in range(4)])

        self.GRU4 = nn.ModuleList([nn.GRU(1, hidden_dim, batch_first=True) for _ in range(3)])
        self.TimeInter4 = nn.ModuleList([TimeTrendAttention(hidden_dim, hidden_dim, hidden_dim, hidden_dim, L=7) for _ in range(3)])

    def forward(self, x):
        """
        x: list of four tensors:
          - x[0]: [B, 9, T]  (1Hz)
          - x[1]: [B, 2, 2T] (2Hz)
          - x[2]: [B, 4, 4T] (4Hz)
          - x[3]: [B, 3, 8T] (8Hz)
        """
        outputs = []
        # helper to process one path
        def process_path(tensor, ms_convs, grus, tts):
            B, N, T = tensor.shape
            outs = []
            for i in range(N):
                # 1) 多尺度卷积 expects [B, C, T], here C=1
                xi = tensor[:, i, :].unsqueeze(1)            # [B, 1, T]
                ci = ms_convs[i](xi)                         # [B, 1, T]
                # 2) GRU 输入 [B, T, C]
                gi, _ = grus[i](ci.permute(0,2,1))           # [B, T, hidden_dim]
                # 3) TimeTrendAttention
                ti, _ = tts[i](gi)                           # [B, T, hidden_dim]
                # 4) 保证长度
                if ti.shape[1] > self.target_len:
                    ti = ti[:, :self.target_len, :]
                outs.append(ti)
            return outs

        # process each frequency path
        outputs += process_path(x[0], self.ms_conv1, self.GRU1, self.TimeInter1)
        outputs += process_path(x[1], self.ms_conv2, self.GRU2, self.TimeInter2)
        outputs += process_path(x[2], self.ms_conv3, self.GRU3, self.TimeInter3)
        outputs += process_path(x[3], self.ms_conv4, self.GRU4, self.TimeInter4)

        # stack 回形状 [B, 18, T, hidden_dim]
        final_out = torch.stack(outputs, dim=1)
        return final_out



    


# ==================== Combined Positional + Trend Diffusion Module ====================
class PositionalTrendDiffusionBlock(nn.Module):
    def __init__(self, max_len, d_model, kernel_size=3, dilation_rates=[1,2,4]):
        super().__init__()
        # 相对位置编码参数
        self.rel_pos = nn.Parameter(torch.randn(max_len, d_model))
        # 趋势扩散分支（多尺度 depth‑wise 卷积）
        self.branches = nn.ModuleList()
        for d in dilation_rates:
            padding = (kernel_size - 1) * d // 2
            self.branches.append(
                nn.Sequential(
                    nn.Conv1d(d_model, d_model, kernel_size,
                              padding=padding, dilation=d,
                              groups=d_model),
                    nn.BatchNorm1d(d_model),
                    nn.GELU()
                )
            )
        self.merge = nn.Conv1d(d_model, d_model, kernel_size=1)

    def forward(self, x):
        # x: [B, T, D]
        B, T, D = x.shape
        # 1) 添加相对位置编码
        pos = self.rel_pos[:T, :]             # [T, D]
        x = x + pos.unsqueeze(0)              # [B, T, D]
        # 2) 转到通道优先进行 depth‑wise 卷积
        x_c = x.permute(0, 2, 1)              # [B, D, T]
        # 3) 多尺度分支融合
        out = sum(branch(x_c) for branch in self.branches) / len(self.branches)
        out = self.merge(out)                 # [B, D, T]
        # 4) 恢复形状
        return out.permute(0, 2, 1)           # [B, T, D]


class ScaledDotProductAttention(nn.Module):
    def __init__(self, scale, attn_drop=0.1):
        super().__init__()
        self.scale = scale
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(attn_drop)

    def forward(self, q, k, v, mask=None):
        scores = torch.matmul(q, k.transpose(-2, -1)) / self.scale
        if mask is not None:
            scores = scores.masked_fill(mask, -1e9)
        attn = self.dropout(self.softmax(scores))
        output = torch.matmul(attn, v)
        return scores, output

class GaussianEnhancedAttention(nn.Module):
    def __init__(self, d_model, win_size, gamma=0.01, attention_dropout=0.1, output_attention=False):
        super().__init__()
        self.output_attention = output_attention
        self.gamma = gamma
        self.win_size = win_size
        self.d_model = d_model

        # 投影层
        self.query_proj = nn.Linear(d_model, d_model)
        self.key_proj = nn.Linear(d_model, d_model)
        self.value_proj = nn.Linear(d_model, d_model)

        # 可学习位置参数 a_i 和 b_i
        self.proj_a = nn.Linear(d_model, 1)
        self.proj_b = nn.Linear(d_model, 1)

        # 可学习距离缩放因子 s
        self.dist_scale = nn.Parameter(torch.tensor(0.2))  # 初始缩放小些以增强局部

        # Scaled Dot-Product Attention
        self.d_k = d_model
        self.scaled_attn = ScaledDotProductAttention(scale=math.sqrt(self.d_k), attn_drop=attention_dropout)

        self.dropout = nn.Dropout(attention_dropout)

        # 初始化距离矩阵 [T, T]
        distances = torch.zeros((win_size, win_size))
        for i in range(win_size):
            for j in range(win_size):
                distances[i, j] = abs(i - j)
        self.register_buffer("distances", distances)

    def forward(self, x):
        """
        x: [B, T, D]
        Returns: 
            out: [B, T, D]
            attn_weights: [B, T, T] (可视化用)
        """
        B, T, D = x.shape
        assert T == self.win_size, f"Expected T == win_size ({self.win_size}), got {T}"

        # --- Bell Decay Part ---
        a = torch.sigmoid(self.proj_a(x)) + 1e-5  # [B, T, 1]
        b = torch.sigmoid(self.proj_b(x)) + 1e-5  # [B, T, 1]
        a = a.expand(-1, -1, T)                   # [B, T, T]
        b = b.expand(-1, -1, T)                   # [B, T, T]
        scaled_dist = self.dist_scale * self.distances  # [T, T]
        dist_view = scaled_dist.unsqueeze(0).expand(B, -1, -1)  # [B, T, T]
        bell_decay = a * torch.exp(-b * dist_view ** 2)         # [B, T, T]

        # --- SDPA Part ---
        Q = self.query_proj(x)  # [B, T, D]
        K = self.key_proj(x)
        V = self.value_proj(x)

        raw_scores, _ = self.scaled_attn(Q.unsqueeze(1), K.unsqueeze(1), V.unsqueeze(1))  # [B, 1, T, T]
        attn_scores = torch.exp(raw_scores.squeeze(1))  # [B, T, T]

        # --- Multiply Fusion ---
        combined_scores = attn_scores * bell_decay + self.gamma  # [B, T, T]
        attn_weights = combined_scores / combined_scores.sum(dim=-1, keepdim=True)  # Normalize
        attn_weights = self.dropout(attn_weights)

        # --- Weighted sum ---
        out = torch.bmm(attn_weights, V)  # [B, T, D]

        if self.output_attention:
            return out, attn_weights
        else:
            return out, None

   

"""
残差连接 经过Elastic Bell Attention机制之后 在经过残差连接和layernorm
"""
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self,d_model,dropout):
        super(SublayerConnection, self).__init__()
        #这里的d_model即经过多头注意力之后的d_model
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        #这里的sublayer即传入的层 这里指MultiHeadedAttention 
        cal_value = sublayer(self.layer_norm(x))
        return x + self.dropout(cal_value)
"""
用于抑制某些时刻的输入
"""
class GLU(nn.Module):
    #Gated Linear Unit
    def __init__(self, input_size):
        super(GLU, self).__init__()
        
        self.fc1 = nn.Linear(input_size,input_size)
        self.fc2 = nn.Linear(input_size, input_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # x [batch,time_len]
        sig = self.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return torch.mul(sig, x) #对两个张量进行逐元素（element-wise）的乘法操作

class GatedResidualNetwork(nn.Module):
    def __init__( self,input_size, hidden_size, output_size,drop_prob=0.1):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.drop_prob = drop_prob
        #这里的input_size和output_size一样
        self.SublayerConnection = SublayerConnection(self.output_size, dropout = self.drop_prob)

        self.gnr = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ELU(),
          GLU(self.hidden_size),
          nn.Linear(self.hidden_size, self.output_size)
        )

    def forward(self, x):
        # x [batch,d_model]
        out = self.SublayerConnection(x,lambda x:self.gnr(x))
        # out [batch,output_size]
        return out
    
class VariableSelectionAttn(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_feats, scale_feats, drop_prob=0.1, share_type="fc"):
        super(VariableSelectionAttn, self).__init__()
        self.n_feats = n_feats
        self.scale_feats = scale_feats 
        self.share_type = share_type

        self.sigmoid = nn.Sigmoid()

        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.drop_prob = drop_prob

        self.avg_pool = nn.AdaptiveAvgPool1d(1)  
        self.max_pool = nn.AdaptiveMaxPool1d(1)  

        self.GNRs = clones(
            GatedResidualNetwork(self.input_size, self.hidden_size, self.output_size, self.drop_prob),
            self.n_feats
        )

        if share_type == "fc":
            self.sharedMlp = nn.Sequential(
                nn.Linear(self.n_feats, self.scale_feats, bias=False),
                nn.ReLU(inplace=True),
                nn.Linear(self.scale_feats, self.n_feats, bias=False),
            )
        else:
            self.sharedMlp = nn.Sequential(
                nn.Conv1d(self.n_feats, self.scale_feats, kernel_size=1),
                nn.ReLU(inplace=True),
                nn.Conv1d(self.scale_feats, self.n_feats, kernel_size=1),
            )

    def forward(self, x):
        if x.dim() == 4:
            B, N, T, D = x.shape
            x = x.permute(0, 2, 1, 3).reshape(B * T, N, D)

            avg_pooled = self.avg_pool(x)
            max_pooled = self.max_pool(x)

            if self.share_type == "fc":
                avg_out = self.sharedMlp(avg_pooled.squeeze(-1))
                max_out = self.sharedMlp(max_pooled.squeeze(-1))
            else:
                avg_out = self.sharedMlp(avg_pooled).squeeze(-1)
                max_out = self.sharedMlp(max_pooled).squeeze(-1)

            weight = self.sigmoid(avg_out + max_out)  # [B*T, N]

            out = 0
            for i in range(self.n_feats):
                gnr_out = self.GNRs[i](x[:, i, :])
                out += weight[:, i].unsqueeze(-1) * gnr_out

            out = out.view(B, T, D)
            self.last_attn = weight.view(B, T, -1)  # [B, T, N]
            return out
        else:
            B, N, D = x.shape

            avg_pooled = self.avg_pool(x)
            max_pooled = self.max_pool(x)

            if self.share_type == "fc":
                avg_out = self.sharedMlp(avg_pooled.squeeze(-1))
                max_out = self.sharedMlp(max_pooled.squeeze(-1))
            else:
                avg_out = self.sharedMlp(avg_pooled).squeeze(-1)
                max_out = self.sharedMlp(max_pooled).squeeze(-1)

            weight = self.sigmoid(avg_out + max_out)

            out = 0
            for i in range(self.n_feats):
                gnr_out = self.GNRs[i](x[:, i, :])
                out += weight[:, i].unsqueeze(-1) * gnr_out

            self.last_attn = weight.unsqueeze(1).expand(-1, x.shape[1], -1)  # [B, T, N]
            return out



class SharedAndTaskSpecificVSA(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_feats, scale_feats, drop_prob=0.1):
        super().__init__()
        self.shared_vsa = VariableSelectionAttn(
            input_size, hidden_size, output_size,
            n_feats, scale_feats, drop_prob, share_type="fc"
        )
        self.task1_vsa = VariableSelectionAttn(
            input_size, hidden_size, output_size,
            n_feats, scale_feats, drop_prob, share_type="fc"
        )
        self.task2_vsa = VariableSelectionAttn(
            input_size, hidden_size, output_size,
            n_feats, scale_feats, drop_prob, share_type="fc"
        )

    def forward(self, x):
        # x: [B, N, T, D]
        shared_out = self.shared_vsa(x)          # [B, T, D]
        task1_out = self.task1_vsa(x)            # [B, T, D]
        task2_out = self.task2_vsa(x)            # [B, T, D]

        shared_attn = self.shared_vsa.last_attn  # [B, T, N]
        task1_attn = self.task1_vsa.last_attn
        task2_attn = self.task2_vsa.last_attn

        out1 = shared_out + task1_out
        out2 = shared_out + task2_out

        return out1, out2, shared_attn, task1_attn, task2_attn


# === 替换原 VCEandVSM_withEBA 模块 ===
class VCEandVSM_withContrastiveVSA(nn.Module):
    def __init__(self, d_model, drop_prob, n_features, scale_feats, max_time_len):
        super().__init__()
        self.bell_attn = GaussianEnhancedAttention(
            d_model=d_model,
            win_size=max_time_len,
            gamma=0.01,
            attention_dropout=drop_prob,
            output_attention=True
        )

        self.attention_unit = SharedAndTaskSpecificVSA(
            input_size=d_model,
            hidden_size=d_model,
            output_size=d_model,
            n_feats=n_features,
            scale_feats=scale_feats,
            drop_prob=drop_prob
        )

    def forward(self, x_stg):
        B, N, T, D = x_stg.shape
        x_flat = x_stg.view(B * N, T, D)
        bell_out, bell_attn_w = self.bell_attn(x_flat)
        x_attn = bell_out.view(B, N, T, D)

        out1, out2, shared_attn, task1_attn, task2_attn = self.attention_unit(x_attn)
        return out1, out2, shared_attn, task1_attn, task2_attn, bell_attn_w.view(B, N, T, T)


# === 对比损失封装 ===
def VSA_contrastive_loss(shared1, shared2, task1, task2, lambda_sim=1.0, lambda_diff=0.5):
    loss_sim = F.mse_loss(shared1, shared2)
    cos_sim = F.cosine_similarity(task1, task2, dim=-1)  # [B, T]
    loss_diff = (1.0 - cos_sim).mean()
    return lambda_sim * loss_sim + lambda_diff * loss_diff





In [14]:
class Mutli_TRBET(nn.Module):
    def __init__(self, num_heads, hidden_dim, drop_prob, n_features, scale_feats, output_size=2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.STG = MTAEncoder(hidden_dim)
        self.pos_trend = PositionalTrendDiffusionBlock(
            max_len=30, d_model=hidden_dim,
            kernel_size=3, dilation_rates=[1,2,4]
        )
        self.VV = VCEandVSM_withContrastiveVSA(
            d_model=hidden_dim,
            drop_prob=drop_prob,
            n_features=n_features,
            scale_feats=scale_feats,
            max_time_len=30
        )
        self.classifier1 = nn.Linear(hidden_dim, output_size)
        self.classifier2 = nn.Linear(hidden_dim, output_size)

    def forward(self, X_1HZ, X_2HZ, X_4HZ, X_8HZ, return_var_attn=False):
        x_list = [X_1HZ, X_2HZ, X_4HZ, X_8HZ]
        x_stg = self.STG(x_list)               # [B, N, T, D]
        B, N, T, D = x_stg.shape
        x = x_stg.view(B * N, T, D)
        x = self.pos_trend(x)
        x_stg = x.view(B, N, T, D)

        out1, out2, shared_attn, task1_attn, task2_attn, bell_attn = self.VV(x_stg)

        feat1 = out1.mean(dim=1)
        feat2 = out2.mean(dim=1)
        out_task1 = self.classifier1(feat1)
        out_task2 = self.classifier2(feat2)

        if return_var_attn:
            return out_task1, out_task2, shared_attn, task1_attn, task2_attn
        return out_task1, out_task2


# 非对称损失函数

In [15]:
# # 非对称损失函数
# import torch
# import torch.nn as nn

# class AsymmetricLossSingleLabel(nn.Module):
#     def __init__(self, gamma_neg=3, gamma_pos=1, clip=0.02, eps=1e-5, pos_weight=None):
#         super().__init__()
#         self.gamma_neg = gamma_neg
#         self.gamma_pos = gamma_pos
#         self.clip = clip
#         self.eps = eps
#         self.pos_weight = pos_weight  # 新增正样本权重参数

#     def forward(self, logits, targets):
#         # 转换标签为概率形式
#         targets_onehot = torch.zeros_like(logits).scatter_(1, targets.unsqueeze(1), 1)

#         # 计算softmax概率
#         probs = torch.softmax(logits, dim=1)
        
#         # 更温和的非对称裁剪
#         if self.clip is not None:
#             probs_neg = torch.clamp(1 - probs[:, 1] + self.clip, max=1)
#             probs = torch.stack([probs_neg, probs[:, 1]], dim=1)

#         # 带权重的交叉熵
#         log_probs = torch.log(probs.clamp(min=self.eps))
#         ce_loss = - (targets_onehot * log_probs).sum(dim=1)
        
#         # 动态调整权重（核心改进）
#         pt = (probs * targets_onehot).sum(dim=1)
#         weights = torch.where(targets == 1,
#                             self.pos_weight * torch.pow(1 - pt, self.gamma_pos),
#                             torch.pow(1 - (1 - pt), self.gamma_neg))
        
#         return (ce_loss * weights).mean()
class AsymmetricLossSingleLabel(nn.Module):
    def __init__(self, gamma_neg=3, gamma_pos=1, clip=0.05, eps=1e-8, 
                 pos_weight=None, label_smoothing=0.1, focal_reduction='mean'):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip  # 稍微放宽裁剪范围
        self.eps = eps    # 更小的数值稳定性项
        self.pos_weight = pos_weight
        self.label_smoothing = label_smoothing
        self.focal_reduction = focal_reduction  # 添加损失缩减选项
        
        # 参数验证
        assert focal_reduction in ['mean', 'sum', 'none'], "reduction must be one of 'mean', 'sum', 'none'"
        assert 0 <= label_smoothing < 1, "label_smoothing must be between 0 and 1"

    def forward(self, logits, targets):
        num_classes = logits.size(1)
        
        # 应用标签平滑
        targets_onehot = torch.zeros_like(logits).scatter_(1, targets.unsqueeze(1), 1)
        if self.label_smoothing > 0:
            targets_onehot = targets_onehot * (1 - self.label_smoothing) + self.label_smoothing / num_classes
        
        # 计算softmax概率
        log_probs = F.log_softmax(logits, dim=1)
        probs = torch.exp(log_probs)
        
        # 改进的非对称概率裁剪
        if self.clip is not None:
            # 对正负类分别裁剪，保持概率总和为1
            probs_pos = torch.clamp(probs[:, 1], min=self.clip, max=1-self.clip)
            probs_neg = 1 - probs_pos
            probs = torch.stack([probs_neg, probs_pos], dim=1)
        
        # 计算交叉熵损失（更稳定的实现）
        ce_loss = - (targets_onehot * log_probs).sum(dim=1)
        
        # 动态调整权重（改进的权重计算）
        pt = (probs * targets_onehot).sum(dim=1)
        weights = torch.where(targets == 1,
                            self.pos_weight * torch.pow(1 - pt + self.eps, self.gamma_pos),
                            torch.pow(pt + self.eps, self.gamma_neg))
        
        # 应用权重
        loss = ce_loss * weights
        
        # 灵活的损失缩减方式
        if self.focal_reduction == 'mean':
            return loss.mean()
        elif self.focal_reduction == 'sum':
            return loss.sum()
        else:  # 'none'
            return loss

## 设置模型参数

In [16]:
#设置param_sizes、namelist
param_sizes = [51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 
               201, 201, 201, 51, 51, 201, 201, 51, 101, 401, 201, 401, 51, 101]
namelist = ['ALT_STD','RADIO_LH','RADIO_RH','IVV','IAS','LDGNOS','PITCH','PITCH_CMD','ROLL','ROLL_CMD','RUDD',
      'WIN_CRS','WIN_ALG','N11','N12','TLA1','TLA2','VRTG']#17个参数
out_path = "/mnt/sdb/Dataset/LiCX/"
model_save = "/mnt/sdb/Dataset/LiCX/" # 存储训练好的模型

epochs = 30

attention_dim = 64#attention的维度
context_dim = 64#上下文向量的维度

server = 75
# 假设输入参数
attention_input_dim = 64  # 必须能被 K=4 整除
attention_dim = 64        # 与 attention_input_dim 相同
context_dim = 64
K = 4
kernel_size = 3

# 多头注意力机制头数
num_heads=4
n_features = len(namelist)
interval_kernel_size = [1 for i in range(n_features)]
#     interval_kernel_size = [2 for i in range(10)] + [3 for i in range(8)]
#     attention_type='step' #选用哪种attention   step/interval
attention_type='interval' #
outchannels = 64 #interval attention 卷积核数
down_output_channels = 32# down sample 映射的维度卷积核数

map_type='linear' #映射qkv 线性映射/卷积映射
is_attention = True
num_classes = 2
# 多头注意力机制中的维度
d_model= 64 #d_model=context_dim
#feedforward
forward_linear_size = 32 #前馈神经的维度
share_type = "fc" #shareMLP的类型 fc/conv

#模型训练参数
batch_size= 256
num_epochs = 30
#保存模型当前epoch的权重
current_w = 'current_w.pth'
#保存最佳的权重
best_w = 'best_w.pth'
#保存模型的文件夹
ckpt = 'ckpt'

average=False
relate = False




sys.path.append("../../")
parser = argparse.ArgumentParser(description='Sequence Modeling - Safety incidents')
parser.add_argument('--cuda', action='store_false',
                    help='use CUDA (default: True)')
parser.add_argument('--clip', type=float, default=-1,
                    help='gradient clip, -1 means no clip (default: -1)')
parser.add_argument('--log-interval', type=int, default=25, metavar='N',
                    help='report interval (default: 100')
parser.add_argument('--seed', type=int, default=42,
                    help='random seed (default: 1111)')
parser.add_argument('--lr', type=float, default=1e-2,
                    help='initial learning rate (default: 1e-4)')
parser.add_argument('--optim', type=str, default='Adam',
                    help='optimizer to use (default: Adam)')
args = parser.parse_known_args()[0]

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

# 重着陆任务（样本更少，需更保守的参数）
loss_func_task1 = AsymmetricLossSingleLabel(
    gamma_neg=3,  # 原5→3（降低对负样本的抑制）
    gamma_pos=1,  # 原3→1（避免过拟合正样本）
    clip=0.02,    # 原0.01→0.02（放宽裁剪）
    eps=1e-5,
    pos_weight=20.0  # 新增正样本权重（约37211/282≈132，取保守值20）
)

# 擦机尾任务（样本稍多，保持适度关注）
loss_func_task2 = AsymmetricLossSingleLabel(
    gamma_neg=2,  # 原4→2（大幅降低抑制）
    gamma_pos=1,  # 原2→1（平衡学习）
    clip=0.05,    # 原0.03→0.05（放宽裁剪）
    eps=1e-5,
    pos_weight=10.0  # 约37211/513≈72，取保守值10
)


## 多任务交互模块

In [17]:
class MultiTaskWeighting(nn.Module):
    """
    一个用于多任务加权的简单封装：
      - 内部保存两个可学习的对数权重 log_alpha1, log_alpha2
      - forward(loss1, loss2) 时，返回 total_loss = alpha1*loss1 + alpha2*loss2
    """
    def __init__(self, init_alpha1=1.0, init_alpha2=1.0):
        super(MultiTaskWeighting, self).__init__()
        # 用对数形式存储，可避免出现负值
        self.log_alpha1 = nn.Parameter(torch.log(torch.tensor(init_alpha1, dtype=torch.float32)))
        self.log_alpha2 = nn.Parameter(torch.log(torch.tensor(init_alpha2, dtype=torch.float32)))

    def forward(self, loss_task1, loss_task2):
        alpha1 = torch.exp(self.log_alpha1)
        alpha2 = torch.exp(self.log_alpha2)
        total_loss = alpha1 * loss_task1 + alpha2 * loss_task2
        return total_loss, alpha1, alpha2
    
# === 完整的交互式多任务包装器 ===
class MultiTaskInteractiveModule(nn.Module):
    def __init__(self, base_model, init_alpha1=0.4, init_alpha2=0.6):
        super().__init__()
        self.base_model = base_model
        self.weight_module = MultiTaskWeighting(init_alpha1, init_alpha2)

    def forward(self, input_1HZ, input_2HZ, input_4HZ, input_8HZ,
                y_task1=None, y_task2=None, loss_func_task1=None, loss_func_task2=None,
                compute_contrastive=False):
        out_task1, out_task2, shared_attn, task1_attn, task2_attn = self.base_model(
            input_1HZ, input_2HZ, input_4HZ, input_8HZ, return_var_attn=True)

        total_loss = None
        alpha1_val = None
        alpha2_val = None
        contrastive_loss = None

        if (y_task1 is not None) and (y_task2 is not None) \
           and (loss_func_task1 is not None) and (loss_func_task2 is not None):
            loss_task1 = loss_func_task1(out_task1, y_task1)
            loss_task2 = loss_func_task2(out_task2, y_task2)
            total_loss, alpha1_val, alpha2_val = self.weight_module(loss_task1, loss_task2)

            if compute_contrastive:
                contrastive_loss = VSA_contrastive_loss(shared_attn, shared_attn, task1_attn, task2_attn)
                total_loss = total_loss + contrastive_loss

        return out_task1, out_task2, total_loss, alpha1_val, alpha2_val, contrastive_loss



In [18]:
# 原先的多任务模型
base_model = Mutli_TRBET(
    num_heads=4,
    hidden_dim=64,  # 必须能被num_heads整除
    drop_prob=0.2,
    n_features=18,
    scale_feats=8,
    output_size=2
).cuda()


# 在其基础上，包装可学习权重
model = MultiTaskInteractiveModule(
    base_model, 
    init_alpha1=0.4,  # 也可调其它初值
    init_alpha2=0.6
).cuda()


In [19]:
# for batch_index, batch_data in enumerate(train_loader):
#     batch_size = batch_data[0].shape[0]
#     num_1HZ = batch_data[0].shape[1]
#     num_2HZ = batch_data[1].shape[1]
#     num_4HZ = batch_data[2].shape[1]
#     num_8HZ = batch_data[3].shape[1]
#     input_1HZ = batch_data[0].cuda()
#     input_2HZ = batch_data[1].reshape(batch_size, num_2HZ, select*2).cuda()
#     input_4HZ = batch_data[2].reshape(batch_size, num_4HZ, select*4).cuda()
#     input_8HZ = batch_data[3].reshape(batch_size, num_8HZ, select*8).cuda()
#     print(f"input_1HZ shape: {input_1HZ.shape}")
#     print(f"input_2HZ shape: {input_2HZ.shape}")
#     print(f"input_4HZ shape: {input_4HZ.shape}")
#     print(f"input_8HZ shape: {input_8HZ.shape}")
#     print(f"Y shape: {batch_data[4].shape}")
# for batch_index, batch_data in enumerate(train_loader):
#     y = batch_data[4]
#     print(y.dtype)  # 直接显示张量类型
#     print(y[:, 0].dtype)  # 检查特定列的类型

## 训练测试

In [24]:
def mkdir(path):
    path=path.strip()
    path=path.rstrip("\\")
    path=path.rstrip("/")
    isExists=os.path.exists(path)
    if not isExists:
        os.makedirs(path) 
def safe_div(n, d):
    return n / d if d != 0 else 0.0

select = 30   
    
def train(train_loader, model, epoch):
    global lr, steps
    model.train()
    batch_idx = 1
    train_loss = 0
    steps = 0
    correct_task1, correct_task2 = 0, 0
    tp_task1, tn_task1, fn_task1, fp_task1 = 0, 0, 0, 0
    tp_task2, tn_task2, fn_task2, fp_task2 = 0, 0, 0, 0
    for batch_index, batch_data in enumerate(train_loader):
         # ========== 1) 数据准备 ========== 
        batch_size = batch_data[0].shape[0]
        num_1HZ = batch_data[0].shape[1]
        num_2HZ = batch_data[1].shape[1]
        num_4HZ = batch_data[2].shape[1]
        num_8HZ = batch_data[3].shape[1]
        input_1HZ = batch_data[0].cuda() #(64,9,30*1)
        input_2HZ = batch_data[1].reshape(batch_size, num_2HZ, select*2).cuda()
        input_4HZ = batch_data[2].reshape(batch_size, num_4HZ, select*4).cuda()
        input_8HZ = batch_data[3].reshape(batch_size, num_8HZ, select*8).cuda()
        # print("1Hz input shape =>", batch_data[0].shape)
        y = batch_data[4].cuda()  # y 的形状应为 (batch_size, 2)
        input_1HZ, input_2HZ, input_4HZ, input_8HZ, y = Variable(input_1HZ), Variable(input_2HZ), Variable(input_4HZ), Variable(input_8HZ), Variable(y)
        optimizer = getattr(optim, args.optim)(model.parameters(), lr=lr)
        seq_length = batch_size
        # ========== 2) 前向传播 + 计算多任务损失 ========== 
        optimizer.zero_grad()
        device = torch.cuda.current_device()
        "1.多任务不交互"
        # output = model(input_1HZ, input_2HZ, input_4HZ, input_8HZ)
        # out_task1, out_task2 = output  # 获取两个任务的输出

        # # 计算损失
        # loss_task1 = loss_func_task1(out_task1, y[:, 0].long())  # 重着陆的损失
        # loss_task2 = loss_func_task2(out_task2, y[:, 1].long())  # 擦机尾的损失
        # loss = 0.4*loss_task1 + 0.6*loss_task2  # 总损失
        # loss.backward()
        "2.多任务交互"
        # 注意这里传入各个子任务标签 & 对应的损失函数
        out_task1, out_task2, total_loss, alpha1, alpha2, contrastive_loss = model(
            input_1HZ, input_2HZ, input_4HZ, input_8HZ,
            y_task1=y[:, 0].long(),
            y_task2=y[:, 1].long(),
            loss_func_task1=loss_func_task1,
            loss_func_task2=loss_func_task2,
            compute_contrastive=True  # ✅ 启用变量注意力对比正则
        )

        # ========== 3) 反向传播 + 更新参数 ========== 
        total_loss.backward()
        if args.clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()

       # ========== 4) 统计训练指标 ========== 
        train_loss += total_loss.item() * batch_size
        steps += batch_size

        # 任务1的准确率
        pred_task1 = out_task1.data.max(1, keepdim=True)[1]
        correct_task1 += pred_task1.eq(y[:, 0].data.view_as(pred_task1)).cpu().sum()
        tp_task1 += ((pred_task1.view(-1) == 1) & (y[:, 0] == 1)).cpu().sum()
        tn_task1 += ((pred_task1.view(-1) == 0) & (y[:, 0] == 0)).cpu().sum()
        fn_task1 += ((pred_task1.view(-1) == 0) & (y[:, 0] == 1)).cpu().sum()
        fp_task1 += ((pred_task1.view(-1) == 1) & (y[:, 0] == 0)).cpu().sum()

        # 任务2的准确率
        pred_task2 = out_task2.data.max(1, keepdim=True)[1]
        correct_task2 += pred_task2.eq(y[:, 1].data.view_as(pred_task2)).cpu().sum()
        tp_task2 += ((pred_task2.view(-1) == 1) & (y[:, 1] == 1)).cpu().sum()
        tn_task2 += ((pred_task2.view(-1) == 0) & (y[:, 1] == 0)).cpu().sum()
        fn_task2 += ((pred_task2.view(-1) == 0) & (y[:, 1] == 1)).cpu().sum()
        fp_task2 += ((pred_task2.view(-1) == 1) & (y[:, 1] == 0)).cpu().sum()

        # 如果想在训练中途观察 alpha1, alpha2，可以打印：
        if (batch_index + 1) % 50 == 0:
            print(f"    [Batch {batch_index+1}] total_loss={total_loss.item():.4f}, "
                  f"alpha1={alpha1:.4f}, alpha2={alpha2:.4f}")


    # ========== 5) 计算整轮 epoch 的平均指标 ========== 
    train_loss = train_loss / steps
    accuracy_task1 = correct_task1.item() / steps
    accuracy_task2 = correct_task2.item() / steps

    p_task1 = safe_div(tp_task1.item(), (tp_task1.item() + fp_task1.item()))
    r_task1 = safe_div(tp_task1.item(), (tp_task1.item() + fn_task1.item()))
    F1_task1 = 2 * p_task1 * r_task1 / (p_task1 + r_task1 + 1e-6)

    p_task2 = safe_div(tp_task2.item(), (tp_task2.item() + fp_task2.item()))
    r_task2 = safe_div(tp_task2.item(), (tp_task2.item() + fn_task2.item()))
    F1_task2 = 2 * p_task2 * r_task2 / (p_task2 + r_task2 + 1e-6)

    print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, '
          f'Task1 Acc: {accuracy_task1:.4f} P: {p_task1:.4f} R: {r_task1:.4f} F1: {F1_task1:.4f}, '
          f'Task2 Acc: {accuracy_task2:.4f} P: {p_task2:.4f} R: {r_task2:.4f} F1: {F1_task2:.4f}')
    
def test(test_loader, model, epoch):
    """
    test_loader: 测试集 DataLoader
    model: MultiTaskInteractiveModule 的实例 (内部已经包含 Mutli_TRBET)
    epoch: 当前epoch号(仅用于打印)
    
    注意:
      - 这里仍然会在测试时计算一下“loss”，
        但并不会反向传播，也不会更新可学习参数。
      - 也可以在推理时不计算loss,只输出预测;具体根据需求决定.
    """
    model.eval()  # 测试时将模型置于 eval 模式，禁用dropout等
    test_loss = 0.0
    steps = 0

    correct_task1, correct_task2 = 0, 0
    tp_task1, tn_task1, fn_task1, fp_task1 = 0, 0, 0, 0
    tp_task2, tn_task2, fn_task2, fp_task2 = 0, 0, 0, 0
    
    with torch.no_grad():  # 测试时不需要计算梯度
        for batch_index, batch_data in enumerate(test_loader):
            # ========== 1) 提取输入与标签 ========== 
            batch_size = batch_data[0].shape[0]
            num_1HZ = batch_data[0].shape[1]
            num_2HZ = batch_data[1].shape[1]
            num_4HZ = batch_data[2].shape[1]
            num_8HZ = batch_data[3].shape[1]

            input_1HZ = batch_data[0].cuda()
            input_2HZ = batch_data[1].reshape(batch_size, num_2HZ, select * 2).cuda()
            input_4HZ = batch_data[2].reshape(batch_size, num_4HZ, select * 4).cuda()
            input_8HZ = batch_data[3].reshape(batch_size, num_8HZ, select * 8).cuda()

            y = batch_data[4].cuda()  # y 的形状应为 [batch_size, 2], 包含两个任务的标签
            
            # ========== 2) 前向推理，获得两个任务的预测 & 测试损失 ========== 
            #   调用包装后的模型 forward, 并传入损失函数 => 它会自动算出多任务加权的 test_loss
            out_task1, out_task2, total_loss, alpha1, alpha2, contrastive_loss = model(
                input_1HZ, input_2HZ, input_4HZ, input_8HZ,
                y_task1=y[:, 0].long(),
                y_task2=y[:, 1].long(),
                loss_func_task1=loss_func_task1,
                loss_func_task2=loss_func_task2,
                compute_contrastive=True  # ✅ 启用变量注意力对比正则
            )

            # 累加总loss
            steps += batch_size
            test_loss += total_loss.item() * batch_size

            # ========== 3) 计算两类任务的预测指标 ========== 
            # ---- 任务1
            pred_task1 = out_task1.data.max(1, keepdim=True)[1]   # shape [batch_size,1]
            correct_task1 += pred_task1.eq(y[:, 0].data.view_as(pred_task1)).cpu().sum()

            tp_task1 += ((pred_task1.view(-1) == 1) & (y[:, 0] == 1)).cpu().sum()
            tn_task1 += ((pred_task1.view(-1) == 0) & (y[:, 0] == 0)).cpu().sum()
            fn_task1 += ((pred_task1.view(-1) == 0) & (y[:, 0] == 1)).cpu().sum()
            fp_task1 += ((pred_task1.view(-1) == 1) & (y[:, 0] == 0)).cpu().sum()

            # ---- 任务2
            pred_task2 = out_task2.data.max(1, keepdim=True)[1]
            correct_task2 += pred_task2.eq(y[:, 1].data.view_as(pred_task2)).cpu().sum()

            tp_task2 += ((pred_task2.view(-1) == 1) & (y[:, 1] == 1)).cpu().sum()
            tn_task2 += ((pred_task2.view(-1) == 0) & (y[:, 1] == 0)).cpu().sum()
            fn_task2 += ((pred_task2.view(-1) == 0) & (y[:, 1] == 1)).cpu().sum()
            fp_task2 += ((pred_task2.view(-1) == 1) & (y[:, 1] == 0)).cpu().sum()

    # ========== 4) 计算最终的平均测试损失和各项指标 ========== 
    test_loss = test_loss / steps

    accuracy_task1 = correct_task1.item() / steps
    accuracy_task2 = correct_task2.item() / steps

    # 任务1
    tp_val_1 = tp_task1.item()
    fp_val_1 = fp_task1.item()
    fn_val_1 = fn_task1.item()

    p_task1 = safe_div(tp_val_1, (tp_val_1 + fp_val_1))
    r_task1 = safe_div(tp_val_1, (tp_val_1 + fn_val_1))
    F1_task1 = 0.0
    if (p_task1 + r_task1) > 0:
        F1_task1 = 2 * p_task1 * r_task1 / (p_task1 + r_task1)

    # 任务2
    tp_val_2 = tp_task2.item()
    fp_val_2 = fp_task2.item()
    fn_val_2 = fn_task2.item()

    p_task2 = safe_div(tp_val_2, (tp_val_2 + fp_val_2))
    r_task2 = safe_div(tp_val_2, (tp_val_2 + fn_val_2))
    F1_task2 = 0.0
    if (p_task2 + r_task2) > 0:
        F1_task2 = 2 * p_task2 * r_task2 / (p_task2 + r_task2)

    # ========== 5) 打印测试指标 ========== 
    print(f'Epoch {epoch}, Test Loss: {test_loss:.4f}, '
          f'Task 1 - Acc: {accuracy_task1:.4f}, P: {p_task1:.4f}, R: {r_task1:.4f}, F1: {F1_task1:.4f}, '
          f'Task 2 - Acc: {accuracy_task2:.4f}, P: {p_task2:.4f}, R: {r_task2:.4f}, F1: {F1_task2:.4f}')


In [21]:
# # 绘图
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.utils import plot_model


# # 绘制模型结构图
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [25]:
if __name__ == "__main__":
    for m in range(1):
        if str(server) == "76":
            model = nn.DataParallel(model,device_ids=[0,1])
        lr = args.lr
        optimizer = getattr(optim, args.optim)(model.parameters(), lr=lr)
        if args.cuda:
            model.cuda()
        acc_list_train,acc_list_test,pre_train,pre_test,re_train,re_test,f1_train,f1_test = [],[],[],[],[],[],[],[]
        loss_train,loss_test = [],[]

        for epoch in range(1, epochs+1):
            train(train_loader,model,epoch)
            test(test_loader,model,epoch)
            if epoch % 10 == 0:
                lr /= 10
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
        torch.cuda.empty_cache()

    [Batch 50] total_loss=nan, alpha1=nan, alpha2=nan
    [Batch 100] total_loss=nan, alpha1=nan, alpha2=nan
Epoch 1, Train Loss: nan, Task1 Acc: 0.9141 P: 0.0105 R: 0.0957 F1: 0.0189, Task2 Acc: 0.9191 P: 0.0137 R: 0.0683 F1: 0.0228
Epoch 1, Test Loss: nan, Task 1 - Acc: 0.9954, P: 0.0000, R: 0.0000, F1: 0.0000, Task 2 - Acc: 0.9871, P: 0.0000, R: 0.0000, F1: 0.0000
    [Batch 50] total_loss=nan, alpha1=nan, alpha2=nan
    [Batch 100] total_loss=nan, alpha1=nan, alpha2=nan
Epoch 2, Train Loss: nan, Task1 Acc: 0.9913 P: 0.0000 R: 0.0000 F1: 0.0000, Task2 Acc: 0.9862 P: 0.0000 R: 0.0000 F1: 0.0000
Epoch 2, Test Loss: nan, Task 1 - Acc: 0.9954, P: 0.0000, R: 0.0000, F1: 0.0000, Task 2 - Acc: 0.9871, P: 0.0000, R: 0.0000, F1: 0.0000
    [Batch 50] total_loss=nan, alpha1=nan, alpha2=nan
    [Batch 100] total_loss=nan, alpha1=nan, alpha2=nan
Epoch 3, Train Loss: nan, Task1 Acc: 0.9913 P: 0.0000 R: 0.0000 F1: 0.0000, Task2 Acc: 0.9862 P: 0.0000 R: 0.0000 F1: 0.0000
Epoch 3, Test Loss: nan, T

KeyboardInterrupt: 

In [26]:
model

MultiTaskInteractiveModule(
  (base_model): Mutli_TRBET(
    (STG): MTAEncoder(
      (ms_conv1): ModuleList(
        (0): MultiScale_TemporalConv(
          (branches): ModuleList(
            (0): Sequential(
              (0): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
              (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Sequential(
              (0): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
              (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (2): Sequential(
              (0): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
              (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (3): Seq

: 